In [ ]:
from omegaconf import DictConfig, OmegaConf
import inspect

import shutil
import numpy as np

from pathlib import Path
from dataclasses import asdict
from carps.utils.task import Task

from optbench import functions
from optbench.abstract_function import AbstractFunction
from __future__ import annotations

import shutil
from collections.abc import Callable
from dataclasses import asdict
from pathlib import Path

import numpy as np
from carps.utils.task import InputSpace, FidelitySpace, OptimizationResources, OutputSpace, Task, TaskMetadata, get_search_space_info
from hydra.utils import instantiate
from carps.utils.generate_tasks import get_dict_input_space, get_dict_metadata, get_dict_opt_resources, get_dict_output_space
from omegaconf import DictConfig, OmegaConf


def fullname(o):
    try:
        # if o is a class or function, get module directly
        module = o.__module__
    except AttributeError:
        # then get module from o's class
        module = o.__class__.__module__
    try:
        # if o is a class or function, get name directly
        name = o.__qualname__
    except AttributeError:
        # then get o's class name
        name = o.__class__.__qualname__
    # if o is a method of builtin class, then module will be None
    if module == 'builtins' or module is None:
        return name
    return module + '.' + name

base_path = Path("../optbench/configs/task")

# Problem ids
benchmark_id = "OptBench"

# from YAHPO paper
def get_n_trials_fn(dimension: int) -> int:
    return int(np.ceil(20 + 40 * np.sqrt(dimension)))

######################

# Generate configs for synthetic functions

target_path = base_path / benchmark_id
if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

objective_function_class = "optbench.optbench_problem.OptBenchProblem"

function_classes = [
    c[1]
    for c in inspect.getmembers(functions, inspect.isclass)
    if issubclass(c[1], AbstractFunction) and c[1] != AbstractFunction
]
dimensions = [2, 5, 10]

identifiers = []
for function_class in function_classes:
    for dim in dimensions:
        problem_id = f"{function_class.__name__}-{dim}"
        optimization_resources = OptimizationResources(
            n_trials=get_n_trials_fn(dim),
            time_budget=None,
            n_workers=1,
        )
        objective_function_cfg = DictConfig({
                "_target_": function_class.__module__ + "." + function_class.__name__,
                "dim": dim,
                "seed": "${seed}",  # get the seed from global cfg
            })
        objective_function_cfg_tmp = objective_function_cfg.copy()
        objective_function_cfg_tmp.seed = 234
        objective_function = instantiate(objective_function_cfg_tmp)
        input_space = InputSpace(
            configuration_space=objective_function.configspace,
            fidelity_space=FidelitySpace(
                is_multifidelity=False,
                fidelity_type=None,
                min_fidelity=None,
                max_fidelity=None,
            )
        )
        output_space = OutputSpace(
            n_objectives=1,
            objectives=["quality"],
        )
        task_metadata = TaskMetadata(
            dimensions=dim,
            has_constraints=False,
            domain="synthetic",
            objective_function_approximation="real",
            has_virtual_time=False,
            deterministic=True,
            search_space_n_categoricals=0,
            search_space_n_ordinals=0,
            search_space_n_integers=0,
            search_space_n_floats=dim,
            search_space_has_conditionals=False,
            search_space_has_forbiddens=False,
            search_space_has_priors=False
        )
        cfg = DictConfig({
            "benchmark_id": benchmark_id,
            "task_id": "${task.name}",
            "task": {
                "_target_": "carps.utils.task.Task",
                "name": problem_id,
                "seed": "${seed}",
                "objective_function": objective_function_cfg,
                "input_space": get_dict_input_space(input_space),
                "output_space": get_dict_output_space(output_space),
                "optimization_resources": get_dict_opt_resources(optimization_resources),
                "metadata": get_dict_metadata(task_metadata),
            },
            # "task": asdict(task)

        })
        identifier = f"{function_class.__name__}_{dim}"
        identifiers.append(identifier)
        fn = target_path / (identifier + ".yaml")
        yaml_str = OmegaConf.to_yaml(cfg=cfg)
        yaml_str = "# @package _global_\n" + yaml_str
        fn.write_text(yaml_str)
        print(cfg)

In [ ]:
",".join(identifiers)